In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config("spark.driver.extraClassPath", "E:\IBM\mysql-connector-java-8.0.18.jar").getOrCreate()
       

In [3]:
df = spark.read.csv('Erasmus.csv', header=True)

In [4]:
df.show(5)

+--------------------+-----------------+---------------+--------------------+----------------------+
|   Project Reference|Mobility Duration|Participant Age|Sending Country Code|Receiving Country Code|
+--------------------+-----------------+---------------+--------------------+----------------------+
|2014-1-BE04-KA347...|                2|             18|                  FR|                    BE|
|2014-1-BE04-KA347...|                3|             22|                  FR|                    BE|
|2014-1-BE04-KA347...|                3|             27|                  FR|                    BE|
|2014-1-BE04-KA347...|                3|             27|                  FR|                    BE|
|2014-1-BE04-KA347...|                3|             25|                  FR|                    BE|
+--------------------+-----------------+---------------+--------------------+----------------------+
only showing top 5 rows



Print number of students that BE received from each country (filter just for one country)

In [5]:
# `` are needed because the colum name has spaces
# % is needed because we want only the BE country, not something that contains BE
df.filter("`Receiving Country Code` like '%BE%'").groupby("Sending Country Code").count().show()


+--------------------+-----+
|Sending Country Code|count|
+--------------------+-----+
|                  LT|    5|
|                  FI|    3|
|                  UA|    1|
|                  RO|    6|
|                  NL|   29|
|                  PL|   31|
|                  MK|    1|
|                  EE|    8|
|                  AT|   33|
|                  RU|    1|
|                  HR|   20|
|                  CZ|    6|
|                  PT|    4|
|                  MT|    2|
|                  DE|   65|
|                  ES|   12|
|                  TR|    2|
|                  ME|    1|
|                  RS|    1|
|                  FR|   33|
+--------------------+-----+
only showing top 20 rows



Print the number of students that each country received from each other country 

In [6]:
df.groupBy("Receiving Country Code", "Sending Country Code").count().orderBy("Receiving Country Code").show()

+----------------------+--------------------+-----+
|Receiving Country Code|Sending Country Code|count|
+----------------------+--------------------+-----+
|                    AT|                  SI|    4|
|                    AT|                  IT|   14|
|                    AT|                  BG|    4|
|                    AT|                  EL|    9|
|                    AT|                  HR|   12|
|                    AT|                  BE|    7|
|                    AT|                  DE|   15|
|                    AT|                  HU|   13|
|                    AT|                  ES|    9|
|                    AT|                  LB|    6|
|                    AT|                  CZ|   12|
|                    AT|                  XK|    1|
|                    AT|                  MK|    1|
|                    AT|                  TR|    4|
|                    AT|                  UA|    5|
|                    AT|                  RO|    5|
|           

In [7]:
df2 = df.groupBy("Receiving Country Code", "Sending Country Code").count().orderBy("Receiving Country Code")
df2.show(df2.count()) # show the entire dataframe

+----------------------+--------------------+-----+
|Receiving Country Code|Sending Country Code|count|
+----------------------+--------------------+-----+
|                    AT|                  BG|    4|
|                    AT|                  HR|   12|
|                    AT|                  BE|    7|
|                    AT|                  DE|   15|
|                    AT|                  EL|    9|
|                    AT|                  SI|    4|
|                    AT|                  HU|   13|
|                    AT|                  ES|    9|
|                    AT|                  IT|   14|
|                    AT|                  LB|    6|
|                    AT|                  CZ|   12|
|                    AT|                  XK|    1|
|                    AT|                  MK|    1|
|                    AT|                  TR|    4|
|                    AT|                  UA|    5|
|                    AT|                  RO|    5|
|           

In [8]:
df2.write.mode("overwrite") \
  .format("jdbc") \
  .option("driver","com.mysql.cj.jdbc.Driver") \
  .option("url", "jdbc:mysql://localhost:3306/erasmus") \
  .option("dbtable", "student_counts") \
  .option("user", "root") \
  .option("password", "stupidity") \
  .save()